In [1]:
import MDAnalysis as mda
import numpy as np

from pprint import pprint

from metamds.db import retrieve_all
from metamds.db import update_doc
from block_avg import block_average

/home/jonestj1/anaconda3/envs/jonestj2_sand/lib/python3.5/site-packages/MDAnalysis/__init__.py:190: UserWarning: #####
MDAnalysis on python 3 is highly experimental!
It is mostly non functional and dramatically untested.
Use at your own risks!!!

  ''')


In [2]:
collection = "peg_oxy_cap"
cursor = retrieve_all(collection=collection)

In [3]:
for doc in cursor:
    if 'friction' in doc.keys():
        continue
    trr_file = '{}shear.trr'.format(doc['output_dir'])
    
    polymer = doc['polymer_class']
    n_molecules = doc ['n_molecules']
    chain_length = doc['chain_length']
    surface = doc['surface_class']
    pattern = doc['pattern_class']
    normal_force = doc['normal_force(kJ/mol/A)']

    name = doc['name']
    
    try:
        trr = mda.coordinates.TRR.TRRReader(trr_file)
    except IOError:
        continue
        
    fric = []
    toss = 1000
    for frame in trr:
        if frame.time > toss:
            top_forces = frame.forces[:int(frame.n_atoms/2)]
            fric.append(float(np.sum(top_forces[:, 0])))
        last_time = frame.time

    update_doc(existing_doc=doc, added_values={"friction":fric}, collection=collection)

In [4]:
cursor = retrieve_all(collection=collection)
for doc in cursor:
    if 'last_time' in doc.keys():
        continue
    update_doc(existing_doc=doc, added_values={"last_time":50000.0}, collection=collection)